## **Tema 1 - P3. Enunciado**
------------------------------------------------------------------------------
***

Para la viga cargada y apoyada según la figura, determinar los esfuerzos en las secciones con posiciones $x=1$, $x=4$, $x=5$, $x=7$. Tomar el origen de $x$ en el extremo $A$ y ejes $x$ horizontal, $y$ vertical. 

<img src="enunciado.png" width="450">

Datos: $L_1$= 2 m, $L_2$= 2.3 m, $T$= 1250 N$\cdot$ m, $F$= 4000 N, $q$= 3000 N/m.




**Resolución:**
------------------------------------------------------------------------------


**Identificación de los apoyos y reacciones**

Para comenzar, es necesario identificar los apoyos de la barra y las reacciones que tienen asociadas. Estas reacciones aparecen cuando el apoyo impide algún grado de libertad. En este caso tenemos:

- Apoyo A: Apoyo articulado fijo. Este tipo de apoyo permite el giro libre de la barra alrededor del punto A, pero no permite el desplazamiento de este punto en $x$ e $y$. Por lo tanto, aparecerán dos fuerzas de reacción, $R_{A,x}$ y $R_{A,y}$. Estas fuerzas aparecen como reacción en el apoyo al impedir el desplazamiento en $x$ e $y$.
    
- Apoyo B: Apoyo articulado móvil. Este tipo de apoyo permite el giro libre de la barra alrededor del punto B y además el desplazamiento de este punto en el eje $x$, pero no permite el desplazamiento en $y$. Por lo tanto, aparecerán una fuerza de reacción, $R_{B,y}$.

Y entonces ya podemos dibujar el estado completo de cargas sobre la barra: solicitaciones, en verde, y reacciones, en rojo:

<img src="reacc.png" width="450">


**Inicialización**

Lo primero que debemos hacer es cargar las librerías necesarias de Python. También definimos las variables que son "dato" del problema.

In [1]:
from sympy import *             # Librería para trabajo simbólico
import math                     # Librería para utilizar símbolos matemáticos como el número pi, que se escribe como math.pi
import numpy as np              # Librería para poder trabajar con matrices y vectores
import matplotlib.pyplot as plt # Librería para poder dibujar gráficas

x=symbols('x')

L1=2.0 # m
L2=2.3 # m
T=1250.0 # Nm
F=4000.0 # N
q=3000.0 # N/m

**Determinación de las reacciones: equilibrio estático**

El siguiente paso es calcular las reacciones. La primera pregunta que debemos hacernos es: ¿es posible calcular las reacciones? (¿tenemos suficientes ecuaciones para hacerlo?). Como en 2D disponemos de 3 ecuaciones de equilibrio estático ($\sum F_x=0$, $\sum F_y=0$ y $\sum M_z=0$), y en este problema existen 3 reacciones (incógnitas), sí que es posible calcularlas. Por lo tanto, este problema será *isostático*. Si por el contrario existieran más de 3 reacciones, el problema sería *hiperestático* (en 2D).

<img src="equivs.png" width="450">

Para hallar el valor de las reacciones, aplicaremos las ecuaciones de equilibrio estático. Para el sumatorio de momentos, tomaremos el punto A como origen de momentos. Las ecuaciones de equilibrio son:

$$\sum F_x=0 \quad\Rightarrow \quad R_{A,x}=0$$

$$\sum F_y=0 \quad\Rightarrow \quad R_{A,y}-F-F_{eq} + R_{B,y}=0$$

$$\sum M_z=0 \quad\Rightarrow \quad T - F (L_1+L_2)- F_{eq}x_{eq} + R_{B,y}(3L_1+L_2)=0$$

Donde el valor de la  fuerza equivalente $F_{eq}$ será:

$$F_{eq}=\int_{2L_1+L_2}^{3L_1+L_2}q dx$$

In [2]:
Feq=integrate(q, (x, 2*L1+L2, 3*L1+L2))
print("Feq=", Feq,"N")

Feq= 6000.00000000000 N


y $x_{eq}$ es:

$$x_{eq} = \frac{\displaystyle{\int_{2L_1+L_2}^{3L_1+L_2}x q dx} }{\displaystyle{\int_{2L_1+L_2}^{3L_1+L_2}q dx}}$$

In [3]:
xeq=integrate(x*q, (x, 2*L1+L2, 3*L1+L2))/integrate(q, (x, 2*L1+L2, 3*L1+L2))
print("xeq=", xeq,"m")

xeq= 7.30000000000000 m


Para calcular las reacciones aplicaremos las ecuaciones de equilibrio estático global. Formularemos un sistema de 3 ecuaciones $ΣFx=0$, $ΣFy=0$ y $ΣMz=0 $ con 3 incógnitas, $R_{A,x}$, $R_{A,y}$ y $R_{B,y}$ y lo resolveremos utilizando la función *linsolve*.

In [4]:
Rax,Ray,Rby=symbols('Rax Ray Rby') #variables simbólicas: reacciones en apoyos
#-------------------------------------#
#          Equilibrio global
#-------------------------------------#
#Se utiliza la funcion "linsolve" que resuelve un sistema de ecuaciones.
#Añadir las ecuaciones ΣFx=0, ΣFy=0 y ΣMz=0 en la primera, segunda y tercera linea. 
#Nota: Todas las reacciones y solicitaciones se consideran positivos en dirección positiva de los ejes cartesianos.
sol,=linsolve([ 
                 Rax,  #Es la eq.: ΣFx=0  ->  Rax=0
                 Ray - F - Feq + Rby,
                 T - F*(L1+L2) - Feq*xeq + Rby*(3*L1+L2)
             ], 
                (Rax,Ray,Rby)  #Es la lista de incógnitas a resolver
            ) 

Rax=sol[0]
Ray=sol[1]
Rby=sol[2]


print("Las reacciones en A y B son:")
print("Rax=", sol[0] , "N")
print("Ray=", sol[1] , "N")
print("Rby=", sol[2], "N")

Las reacciones en A y B son:
Rax= 0 N
Ray= 2801.20481927711 N
Rby= 7198.79518072289 N


**Determinación de esfuerzos internos: aplicación del método de las secciones (equilibrio elástico)**

Para determinar los esfuerzos dentro de la barra (el denominado torsor de esfuerzos), en particular el esfuerzo cortante, el esfuerzo normal y el momento flector, es necesario aplicar el método de las secciones. Habrá que cortar la barra por los puntos de interés y aplicar a una de las dos mitades las ecuaciones de equilibrio elástico.

Empezaremos por el nudo A y seguiremos con los nudos C, D y E.

**Tramo 1: punto $x=1$**

Para analizar los esfuerzos en el tramo 1, cortamos la barra por una sección arbitraria localizada a lo largo del tramo 1 ($0<x<L_1$) como se muestra en la figura. 

<img src="corte1.png" width="500">

Para todo el tramo 1 ($0<x<L_1$), tendremos las mismas expresiones de los esfuerzos, ya que a lo largo de este tramo no hay fuerzas y momentos externos.

El siguiente paso es representar el diagrama de esfuerzos para una de las dos mitades que separa el plano de corte. En este caso nos quedaremos con la mitad izquierda (será más sencillo, ya que sólo hay 2 reacciones, mientras que en el lado derecho hay muchas más fuerzas y momentos). Este diagrama se representa en la siguiente figura:

<img src="elas1.png" width="160">

Ahora, para hallar $N_x$, $V_y$ y $M_z$ se utilizarán las ecuaciones de equilibrio elástico: $\sum F_{x,exteriores} + \sum F_{x,interiores}=0$, $\sum F_{y,exteriores} + \sum F_{y,interiores}=0$ y $\sum M_{z,exteriores} + \sum M_{z,interiores}=0$, donde las fuerzas y momentos interiores ($N_x$, $V_y$ y $M_z$) serán las incógnitas y las exteriores (solicitaciones y reacciones) serán conocidas. El equilibrio de momentos se puede realizar respecto al origen (A) o a la sección de corte. En este caso lo haremos respecto a la sección de corte. Las ecuaciones de equilibrio elástico son:

$$\sum F_x=0 \quad\Rightarrow \quad R_{A,x}+N_x=0$$

$$\sum F_y=0 \quad\Rightarrow \quad R_{A,y}+V_y=0$$

$$\sum M_z =0 \quad\Rightarrow \quad M_z-R_{A,y} x =0$$


In [5]:
Nx,Vy,Mz=symbols('Nx Vy Mz') #variables simbólicas: torsor esfuerzos
xp=1.0 #sección corte

sol,=linsolve([ 
                 Rax + Nx,  
                 Ray + Vy,
                 Mz - Ray*xp
             ], 
                (Nx,Vy,Mz)  #Es la lista de incógnitas a resolver
            ) 

print("Los esfuerzos en la sección son:")
print("Nx=", sol[0] , "N")
print("Vy=", sol[1], "N")
print("Mz=", sol[2] , "Nm")

Los esfuerzos en la sección son:
Nx= 0 N
Vy= -2801.20481927711 N
Mz= 2801.20481927711 Nm


**Tramo 2: punto $x=4$**

Como en el caso anterior, para hallar los esfuerzos  $N_x$, $V_y$ y $M_z$ en el tramo 2 haremos un corte arbitrariamente en este tramo, entre $x=2$ y $x=4.3$ m. La localización de la sección de corte se muestra en la figura inferior, observándose que las distancias desde el origen hasta dicha sección vienen dadas por $x$.

<img src="corte2.png" width="500">

Se aplicará el equilibrio elástico a la mitad izquierda como se muestra en la figura siguiente. En esta figura se han representado los esfuerzos internos (torsor de esfuerzos) en la sección, los cuales debemos calcular.

<img src="elas2.png" width="250">

Como ejemplo alternativo al del tramo anterior, en este caso el equilibrio de momentos se realizará *respecto a la sección de corte*, aunque también podríamos hacerlo respecto del origen. Las ecuaciones de equilibrio elástico son:


$$  \sum F_x=0 \quad\Rightarrow \quad R_{A,x}+N_x=0$$

$$ \sum F_y=0 \quad\Rightarrow \quad R_{A,y}+V_y=0$$

$$ \sum M_z =0 \quad\Rightarrow \quad -R_{A,y} x  + T + M_z=0$$

In [6]:
Nx,Vy,Mz=symbols('Nx Vy Mz') #variables simbólicas: torsor esfuerzos
xp=4.0 #sección corte

sol,=linsolve([ 
                 Rax + Nx,  
                 Ray + Vy,
                 -Ray*xp + T + Mz
             ], 
                (Nx,Vy,Mz)  #Es la lista de incógnitas a resolver
            ) 

print("Los esfuerzos en la sección son:")
print("Nx=", sol[0] , "N")
print("Vy=", sol[1], "N")
print("Mz=", sol[2] , "Nm")

Los esfuerzos en la sección son:
Nx= 0 N
Vy= -2801.20481927711 N
Mz= 9954.81927710844 Nm


**Tramo 3: punto $x=6$**

Se deja como ejercicio.


**Tramo 4: punto $x=7$**

Como en el caso anterior, para hallar los esfuerzos  $N_x$, $V_y$ y $M_z$ en el tramo 4 haremos un corte arbitrariamente en este tramo, entre $x=6.3$ y $x=8.3$ m. La localización de la sección de corte se muestra en la figura inferior, observándose que las distancias desde el origen hasta dicha sección vienen dadas por $x$.

<img src="corte4.png" width="500">

Se aplicará el equilibrio elástico a la mitad izquierda como se muestra en la figura siguiente. En esta figura se han representado los esfuerzos internos (torsor de esfuerzos) en la sección, los cuales debemos calcular.

<img src="elas4.png" width="500">

El equilibrio de momentos se realizará respecto al origen (punto A). Las ecuaciones de equilibrio elástico son:

$$ \sum F_x=0 \quad\Rightarrow \quad R_{A,x}+N_x=0$$

$$ \sum F_y=0 \quad\Rightarrow \quad R_{A,y}+V_y-F-F_{eq,s} =0$$

$$ \sum M_z =0 \quad\Rightarrow \quad    T - F(L_1+L_2) - F_{eq,s}x_{eq,s} + V_y x + M_z =0$$

**¡¡¡Atención, error frecuente!!!:** $F_{eq,s}$ y $x_{eq,s}$ son la fuerza equivalente y el punto de aplicación de ésta considerando la distribución de carga $q$ **solamente hasta la sección de corte**, es decir, hasta $x_p$, **y no en todo el tramo 4**. El valor de la  fuerza equivalente $F_{eq,s}$ será:

$$F_{eq,s}=\int_{2L_1+L_2}^{x_p}q dx$$

In [7]:
xp=7.0 #definimos ya la sección corte

Feq=integrate(q, (x, 2*L1+L2, xp))
print("Feq=", Feq,"N")

Feq= 2100.00000000000 N


y $x_{eq,s}$ es:

$$x_{eq,s} = \frac{\displaystyle{\int_{2L_1+L_2}^{x_p}x q dx} }{\displaystyle{\int_{2L_1+L_2}^{x_p}q dx}}$$

In [8]:
xeq=integrate(x*q, (x, 2*L1+L2, xp))/integrate(q, (x, 2*L1+L2, xp))
print("xeq=", xeq,"m")

xeq= 6.65000000000000 m


In [9]:
Nx,Vy,Mz=symbols('Nx Vy Mz') #variables simbólicas: torsor esfuerzos

sol,=linsolve([ 
                 Rax + Nx,  
                 Ray + Vy - F -Feq,
                 T - F*(L1+L2) - Feq*xeq + Vy*xp + Mz
             ], 
                (Nx,Vy,Mz)  #Es la lista de incógnitas a resolver
            ) 

print("Los esfuerzos en la sección son:")
print("Nx=", sol[0] , "N")
print("Vy=", sol[1] , "N")
print("Mz=", sol[2] , "Nm")

Los esfuerzos en la sección son:
Nx= 0 N
Vy= 3298.79518072289 N
Mz= 6823.43373493977 Nm
